In [1]:
from model import model,test_iter,train_iter
import torch
from torch import nn
from torch_geometric.transforms import ToUndirected
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

/Users/asadsabir/anaconda3/envs/traffic/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [2]:
loss = nn.MSELoss()
vmse = torch.vmap(loss)
undirect = ToUndirected()

H_t = None
C_t = None
model.eval()
mean_mse_te = 0
for ttime, tsnapshot in enumerate(test_iter):
    tsnapshot = undirect(tsnapshot)
    pred,H_t,C_t = model(tsnapshot.x, tsnapshot.edge_index, H_t,C_t)
    mean_mse_te = mean_mse_te + torch.mean(vmse(pred,tsnapshot.y))

    if ttime % 4 == 0:
        H_t = None
print(mean_mse_te/(ttime+1))


H_t = None
C_t = None
model.eval()
mean_mse_te = 0
for ttime, tsnapshot in enumerate(train_iter):
    tsnapshot = undirect(tsnapshot)
    pred,H_t,C_t = model(tsnapshot.x, tsnapshot.edge_index, H_t,C_t)
    mean_mse_te = mean_mse_te + torch.mean(vmse(pred,tsnapshot.y))

    if ttime % 4 == 0:
        H_t = None
print(mean_mse_te/(ttime+1))

/Users/asadsabir/anaconda3/envs/traffic/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.0021, grad_fn=<DivBackward0>)
tensor(0.0012, grad_fn=<DivBackward0>)


In [3]:
model_dict = model.state_dict()

for k in model_dict.keys():
    print(k)

edge_weight
recurrent.w_c_i
recurrent.b_i
recurrent.w_c_f
recurrent.b_f
recurrent.b_c
recurrent.w_c_o
recurrent.b_o
recurrent.conv_x_i.bias
recurrent.conv_x_i.lins.0.weight
recurrent.conv_x_i.lins.1.weight
recurrent.conv_x_i.lins.2.weight
recurrent.conv_h_i.bias
recurrent.conv_h_i.lins.0.weight
recurrent.conv_h_i.lins.1.weight
recurrent.conv_h_i.lins.2.weight
recurrent.conv_x_f.bias
recurrent.conv_x_f.lins.0.weight
recurrent.conv_x_f.lins.1.weight
recurrent.conv_x_f.lins.2.weight
recurrent.conv_h_f.bias
recurrent.conv_h_f.lins.0.weight
recurrent.conv_h_f.lins.1.weight
recurrent.conv_h_f.lins.2.weight
recurrent.conv_x_c.bias
recurrent.conv_x_c.lins.0.weight
recurrent.conv_x_c.lins.1.weight
recurrent.conv_x_c.lins.2.weight
recurrent.conv_h_c.bias
recurrent.conv_h_c.lins.0.weight
recurrent.conv_h_c.lins.1.weight
recurrent.conv_h_c.lins.2.weight
recurrent.conv_x_o.bias
recurrent.conv_x_o.lins.0.weight
recurrent.conv_x_o.lins.1.weight
recurrent.conv_x_o.lins.2.weight
recurrent.conv_h_o.bias

In [4]:
for k,v in model_dict.items():

    if 'recurrent' in k:
        print(k,v.size())

recurrent.w_c_i torch.Size([1, 32])
recurrent.b_i torch.Size([1, 32])
recurrent.w_c_f torch.Size([1, 32])
recurrent.b_f torch.Size([1, 32])
recurrent.b_c torch.Size([1, 32])
recurrent.w_c_o torch.Size([1, 32])
recurrent.b_o torch.Size([1, 32])
recurrent.conv_x_i.bias torch.Size([32])
recurrent.conv_x_i.lins.0.weight torch.Size([32, 49])
recurrent.conv_x_i.lins.1.weight torch.Size([32, 49])
recurrent.conv_x_i.lins.2.weight torch.Size([32, 49])
recurrent.conv_h_i.bias torch.Size([32])
recurrent.conv_h_i.lins.0.weight torch.Size([32, 32])
recurrent.conv_h_i.lins.1.weight torch.Size([32, 32])
recurrent.conv_h_i.lins.2.weight torch.Size([32, 32])
recurrent.conv_x_f.bias torch.Size([32])
recurrent.conv_x_f.lins.0.weight torch.Size([32, 49])
recurrent.conv_x_f.lins.1.weight torch.Size([32, 49])
recurrent.conv_x_f.lins.2.weight torch.Size([32, 49])
recurrent.conv_h_f.bias torch.Size([32])
recurrent.conv_h_f.lins.0.weight torch.Size([32, 32])
recurrent.conv_h_f.lins.1.weight torch.Size([32, 32]

In [5]:
tsnapshot.edge_index

tensor([[ 0,  0,  0,  0,  1,  1,  2,  2,  3,  4,  4,  4,  4,  4,  5,  5,  6,  6,
          7,  8,  8,  9,  9, 10, 10, 11, 12, 12, 13, 13, 14, 14, 14, 14, 14, 15,
         16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 21, 21, 22, 22, 23, 23,
         24, 24, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 27, 28, 28, 29, 29, 29,
         29, 29, 30, 30, 31, 31, 31, 32, 33, 33, 34, 34, 35, 35],
        [ 1, 14, 25, 29,  0,  2,  1,  3,  2,  5, 21, 25, 26, 31,  4,  6,  5,  7,
          6,  9, 10,  8, 12,  8, 13, 12,  9, 11, 10, 14,  0, 13, 25, 26, 31, 16,
         15, 17, 16, 18, 17, 19, 18, 20, 19, 21,  4, 20, 25, 29, 23, 25, 22, 24,
         23, 25,  0,  4, 14, 21, 22, 24, 29,  4, 14, 29, 28, 27, 35,  0, 21, 25,
         26, 35, 32, 33,  4, 14, 34, 30, 30, 34, 31, 33, 28, 29]])

In [6]:
ew_key = np.array(tsnapshot.edge_index)

In [7]:
model_dict['edge_weight']

tensor([0.8063, 0.9568, 0.7983, 1.0436, 1.0413, 0.9039, 0.9185, 0.9186, 0.9908,
        0.9789, 0.9106, 0.9128, 0.9581, 0.9359, 0.8989, 0.9148, 0.8940, 1.0440,
        0.8018, 1.0115, 0.8845, 0.8612, 0.9274, 0.9752, 1.1028, 0.8548, 0.9568,
        0.9132, 0.8688, 0.8538, 0.9683, 0.9318, 0.8459, 0.7898, 0.8184, 0.7752,
        0.9634, 0.9797, 0.8144, 1.0119, 0.8411, 1.0547, 0.8109, 1.0217, 0.9200,
        0.9072, 0.8969, 0.9580, 0.9110, 1.0119, 0.8651, 0.9551, 0.9744, 0.8140,
        0.9778, 1.0209, 0.9399, 0.8770, 1.0187, 0.7749, 1.0470, 0.8000, 1.1267,
        0.9035, 1.0506, 1.0211, 1.0234, 0.8380, 0.9466, 0.8625, 0.6902, 0.7293,
        0.6943, 1.1502, 0.9500, 0.9230, 0.9013, 0.9252, 0.9636, 0.8611, 0.9070,
        1.0218, 0.8857, 0.8578, 0.9633, 0.8048])

In [8]:
edge_weights = model_dict['edge_weight'].detach().numpy()

In [9]:
avg_ew = []

for i,edge in enumerate(zip(ew_key[0],ew_key[1])):
    for j in range(i+1,86):
        edge2 = (ew_key[0][j],ew_key[1][j])
        if edge[0] == edge2[1] and edge[1] == edge2[0]:
            avg_ew.append(np.mean(edge_weights[[i,j]]))

In [10]:
min(avg_ew)

0.8429203

In [11]:
max(avg_ew)

1.0010267

In [12]:
np.save('average_edge_weights.npy',avg_ew)

In [13]:
import torch
from torch.nn import Parameter
from torch_geometric.nn import ChebConv
from torch_geometric.nn.inits import glorot, zeros


class GConvLSTM_explained(torch.nn.Module):

    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        K: int,
        normalization: str = "sym",
        bias: bool = True,
    ):
        super(GConvLSTM_explained, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.K = K
        self.normalization = normalization
        self.bias = bias
        self._create_parameters_and_layers()
        self._set_parameters()

    def _create_input_gate_parameters_and_layers(self):

        self.conv_x_i = ChebConv(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            K=self.K,
            normalization=self.normalization,
            bias=self.bias,
        )

        self.conv_h_i = ChebConv(
            in_channels=self.out_channels,
            out_channels=self.out_channels,
            K=self.K,
            normalization=self.normalization,
            bias=self.bias,
        )

        self.w_c_i = Parameter(torch.Tensor(1, self.out_channels))
        self.b_i = Parameter(torch.Tensor(1, self.out_channels))

    def _create_forget_gate_parameters_and_layers(self):

        self.conv_x_f = ChebConv(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            K=self.K,
            normalization=self.normalization,
            bias=self.bias,
        )

        self.conv_h_f = ChebConv(
            in_channels=self.out_channels,
            out_channels=self.out_channels,
            K=self.K,
            normalization=self.normalization,
            bias=self.bias,
        )

        self.w_c_f = Parameter(torch.Tensor(1, self.out_channels))
        self.b_f = Parameter(torch.Tensor(1, self.out_channels))

    def _create_cell_state_parameters_and_layers(self):

        self.conv_x_c = ChebConv(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            K=self.K,
            normalization=self.normalization,
            bias=self.bias,
        )

        self.conv_h_c = ChebConv(
            in_channels=self.out_channels,
            out_channels=self.out_channels,
            K=self.K,
            normalization=self.normalization,
            bias=self.bias,
        )

        self.b_c = Parameter(torch.Tensor(1, self.out_channels))

    def _create_output_gate_parameters_and_layers(self):

        self.conv_x_o = ChebConv(
            in_channels=self.in_channels,
            out_channels=self.out_channels,
            K=self.K,
            normalization=self.normalization,
            bias=self.bias,
        )

        self.conv_h_o = ChebConv(
            in_channels=self.out_channels,
            out_channels=self.out_channels,
            K=self.K,
            normalization=self.normalization,
            bias=self.bias,
        )

        self.w_c_o = Parameter(torch.Tensor(1, self.out_channels))
        self.b_o = Parameter(torch.Tensor(1, self.out_channels))

    def _create_parameters_and_layers(self):
        self._create_input_gate_parameters_and_layers()
        self._create_forget_gate_parameters_and_layers()
        self._create_cell_state_parameters_and_layers()
        self._create_output_gate_parameters_and_layers()

    def _set_parameters(self):
        glorot(self.w_c_i)
        glorot(self.w_c_f)
        glorot(self.w_c_o)
        zeros(self.b_i)
        zeros(self.b_f)
        zeros(self.b_c)
        zeros(self.b_o)

    def _set_hidden_state(self, X, H):
        if H is None:
            H = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return H

    def _set_cell_state(self, X, C):
        if C is None:
            C = torch.zeros(X.shape[0], self.out_channels).to(X.device)
        return C

    def _calculate_input_gate(self, X, edge_index, edge_weight, H, C, lambda_max):
        I = self.conv_x_i(X, edge_index, edge_weight, lambda_max=lambda_max)
        I = I + self.conv_h_i(H, edge_index, edge_weight, lambda_max=lambda_max)
        I = I + (self.w_c_i * C)
        I = I + self.b_i
        I = torch.sigmoid(I)
        return I

    def _calculate_forget_gate(self, X, edge_index, edge_weight, H, C, lambda_max):
        F = self.conv_x_f(X, edge_index, edge_weight, lambda_max=lambda_max)
        F = F + self.conv_h_f(H, edge_index, edge_weight, lambda_max=lambda_max)
        F = F + (self.w_c_f * C)
        F = F + self.b_f
        F = torch.sigmoid(F)
        return F

    def _calculate_cell_state(self, X, edge_index, edge_weight, H, C, I, F, lambda_max):
        T = self.conv_x_c(X, edge_index, edge_weight, lambda_max=lambda_max)
        T = T + self.conv_h_c(H, edge_index, edge_weight, lambda_max=lambda_max)
        T = T + self.b_c
        T = torch.tanh(T)
        C = F * C + I * T
        return C

    def _calculate_output_gate(self, X, edge_index, edge_weight, H, C, lambda_max):
        O = self.conv_x_o(X, edge_index, edge_weight, lambda_max=lambda_max)
        O = O + self.conv_h_o(H, edge_index, edge_weight, lambda_max=lambda_max)
        O = O + (self.w_c_o * C)
        O = O + self.b_o
        O = torch.sigmoid(O)
        return O

    def _calculate_hidden_state(self, O, C):
        H = O * torch.tanh(C)
        return H

    def forward(
        self,
        X: torch.FloatTensor,
        edge_index: torch.LongTensor,
        edge_weight: torch.FloatTensor = None,
        H: torch.FloatTensor = None,
        C: torch.FloatTensor = None,
        lambda_max: torch.Tensor = None,
    ) -> torch.FloatTensor:
       
        H = self._set_hidden_state(X, H)
        C = self._set_cell_state(X, C)
        I = self._calculate_input_gate(X, edge_index, edge_weight, H, C, lambda_max)
        F = self._calculate_forget_gate(X, edge_index, edge_weight, H, C, lambda_max)
        C = self._calculate_cell_state(X, edge_index, edge_weight, H, C, I, F, lambda_max)
        O = self._calculate_output_gate(X, edge_index, edge_weight, H, C, lambda_max)
        H = self._calculate_hidden_state(O, C)
        return H, C, O, F, I

In [14]:
from typing import List, Optional, Tuple, Union

import torch.nn.functional as F
from torch import Tensor

from torch_geometric.nn.aggr import Aggregation, MultiAggregation
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.typing import Adj, OptPairTensor, Size, SparseTensor
from torch_geometric.utils import spmm
from typing import (Any)
from torch_geometric.utils import (
    is_sparse
)


class SAGEConv_explained(MessagePassing):
    def __init__(
        self,
        in_channels: Union[int, Tuple[int, int]],
        out_channels: int,
        aggr: Optional[Union[str, List[str], Aggregation]] = "mean",
        normalize: bool = False,
        root_weight: bool = True,
        project: bool = False,
        bias: bool = True,
        **kwargs,
    ):
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize
        self.root_weight = root_weight
        self.project = project

        if isinstance(in_channels, int):
            in_channels = (in_channels, in_channels)

        if aggr == 'lstm':
            kwargs.setdefault('aggr_kwargs', {})
            kwargs['aggr_kwargs'].setdefault('in_channels', in_channels[0])
            kwargs['aggr_kwargs'].setdefault('out_channels', in_channels[0])

        super().__init__(aggr, **kwargs)

        if self.project:
            if in_channels[0] <= 0:
                raise ValueError(f"'{self.__class__.__name__}' does not "
                                 f"support lazy initialization with "
                                 f"`project=True`")
            self.lin = Linear(in_channels[0], in_channels[0], bias=True)

        if isinstance(self.aggr_module, MultiAggregation):
            aggr_out_channels = self.aggr_module.get_out_channels(
                in_channels[0])
        else:
            aggr_out_channels = in_channels[0]

        self.lin_l = Linear(aggr_out_channels, out_channels, bias=bias)
        if self.root_weight:
            self.lin_r = Linear(in_channels[1], out_channels, bias=False)

        self.reset_parameters()

    def reset_parameters(self):
        super().reset_parameters()
        if self.project:
            self.lin.reset_parameters()
        self.lin_l.reset_parameters()
        if self.root_weight:
            self.lin_r.reset_parameters()

    def forward(
        self,
        x: Union[Tensor, OptPairTensor],
        edge_index: Adj,
        size: Size = None,
    ) -> Tensor:

        if isinstance(x, Tensor):
            x = (x, x)

        if self.project and hasattr(self, 'lin'):
            x = (self.lin(x[0]).relu(), x[1])

        # propagate_type: (x: OptPairTensor)
        out,x_j = self.propagate(edge_index, x=x, size=size)

        neighbor_features = {}

        #tsnapshot.edge_index
        #record[1]
        edge_diary = np.array(edge_index)
        message = x_j
        

        for s in range(36):
            neighbor_features[int(s)] = {}
            
            for i,val in enumerate(edge_diary[1]):
                if val == s:
                    neighbor_features[s][int(edge_diary[0][i])] = message[i]
            
            n_neighbors = len(neighbor_features[s].keys())
            for n,v in neighbor_features[s].items():
                v = v/(n_neighbors)
                neighbor_features[s][n] = torch.dot(torch.reshape(self.lin_l.weight,(32,)),v)


        out = self.lin_l(out)

        x_r = x[1]
        
        if self.root_weight and x_r is not None:
            root = self.lin_r(x_r)
            
            for s in range(36):
                neighbor_features[s][s] = root[s]
            out = out + root

        if self.normalize:
            out = F.normalize(out, p=2., dim=-1)
        
        return out,neighbor_features

    def message(self, x_j: Tensor) -> Tensor:
        return x_j

    def message_and_aggregate(self, adj_t: Adj, x: OptPairTensor) -> Tensor:
        if isinstance(adj_t, SparseTensor):
            adj_t = adj_t.set_value(None, layout=None)
        
        return spmm(adj_t, x[0], reduce=self.aggr)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.in_channels}, '
                f'{self.out_channels}, aggr={self.aggr})')
    
    def propagate(
        self,
        edge_index: Adj,
        size: Size = None,
        **kwargs: Any,
    ) -> Tensor:
        
        decomposed_layers = 1 if self.explain else self.decomposed_layers

        for hook in self._propagate_forward_pre_hooks.values():
            res = hook(self, (edge_index, size, kwargs))
            if res is not None:
                edge_index, size, kwargs = res

        mutable_size = self._check_input(edge_index, size)

        # Run "fused" message and aggregation (if applicable).
        if is_sparse(edge_index) and self.fuse and not self.explain:
            coll_dict = self._collect(self._fused_user_args, edge_index,
                                      mutable_size, kwargs)

            msg_aggr_kwargs = self.inspector.collect_param_data(
                'message_and_aggregate', coll_dict)
            for hook in self._message_and_aggregate_forward_pre_hooks.values():
                res = hook(self, (edge_index, msg_aggr_kwargs))
                if res is not None:
                    edge_index, msg_aggr_kwargs = res
            out = self.message_and_aggregate(edge_index, **msg_aggr_kwargs)
            for hook in self._message_and_aggregate_forward_hooks.values():
                res = hook(self, (edge_index, msg_aggr_kwargs), out)
                if res is not None:
                    out = res

            update_kwargs = self.inspector.collect_param_data(
                'update', coll_dict)
            out = self.update(out, **update_kwargs)

        else:  # Otherwise, run both functions in separation.
            
            if decomposed_layers > 1:
                user_args = self._user_args
                decomp_args = {a[:-2] for a in user_args if a[-2:] == '_j'}
                decomp_kwargs = {
                    a: kwargs[a].chunk(decomposed_layers, -1)
                    for a in decomp_args
                }
                decomp_out = []

            for i in range(decomposed_layers):
                if decomposed_layers > 1:
                    for arg in decomp_args:
                        kwargs[arg] = decomp_kwargs[arg][i]

                coll_dict = self._collect(self._user_args, edge_index,
                                          mutable_size, kwargs)

                msg_kwargs = self.inspector.collect_param_data(
                    'message', coll_dict)
                for hook in self._message_forward_pre_hooks.values():
                    res = hook(self, (msg_kwargs, ))
                    if res is not None:
                        msg_kwargs = res[0] if isinstance(res, tuple) else res
                #record.append(msg_kwargs['x_j'].detach().numpy())
                out = self.message(**msg_kwargs)
                for hook in self._message_forward_hooks.values():
                    res = hook(self, (msg_kwargs, ), out)
                    if res is not None:
                        out = res

                if self.explain:
                    explain_msg_kwargs = self.inspector.collect_param_data(
                        'explain_message', coll_dict)
                    out = self.explain_message(out, **explain_msg_kwargs)

                aggr_kwargs = self.inspector.collect_param_data(
                    'aggregate', coll_dict)
                for hook in self._aggregate_forward_pre_hooks.values():
                    res = hook(self, (aggr_kwargs, ))
                    if res is not None:
                        aggr_kwargs = res[0] if isinstance(res, tuple) else res

                out = self.aggregate(out, **aggr_kwargs)
                #record.append(('after aggregate',out.detach().numpy()))
                for hook in self._aggregate_forward_hooks.values():
                    res = hook(self, (aggr_kwargs, ), out)
                    if res is not None:
                        out = res

                update_kwargs = self.inspector.collect_param_data(
                    'update', coll_dict)
                out = self.update(out, **update_kwargs)
                #record.append(('after update',out.detach().numpy()))
                if decomposed_layers > 1:
                    decomp_out.append(out)

            if decomposed_layers > 1:
                out = torch.cat(decomp_out, dim=-1)

        for hook in self._propagate_forward_hooks.values():
            res = hook(self, (edge_index, mutable_size, kwargs), out)
            if res is not None:
                out = res

        return out,msg_kwargs['x_j']

In [15]:
class GLSTM_explained(torch.nn.Module):
    def __init__(self, node_features, filters,num_edges):
        super(GLSTM_explained, self).__init__()
        self.recurrent = GConvLSTM_explained(node_features, filters, 3)
        self.sage = SAGEConv_explained(filters,1,project=True)                           
        self.edge_weight = nn.Parameter(torch.ones(num_edges,dtype=torch.float))
        
    def forward(self, x, edge_index, hidden_state, cell_state):
        
        h,c, o, f, i = self.recurrent(x, edge_index, self.edge_weight,H=hidden_state,C=cell_state)
        
        y = F.relu(h)
        y,na = self.sage(y,edge_index)

        return y,h,c,o,f,i,na

In [16]:
model_explained = GLSTM_explained(49,32,86)
model_explained.load_state_dict(model_dict)

<All keys matched successfully>

In [17]:
H_t = None
C_t = None
LSTM_gates_te = []
neighbor_effects_te = []
target_te = []
model_explained.eval()
mean_mse_te = 0
for ttime, tsnapshot in enumerate(test_iter):
    tsnapshot = undirect(tsnapshot)
    pred,H_t,C_t,o,f,i,na = model_explained(tsnapshot.x, tsnapshot.edge_index, H_t,C_t)

    LSTM_gates_te.append([H_t.detach(), C_t.detach(), o.detach(), f.detach(), i.detach()])
    neighbor_effects_te.append(na)
    target_te.append(pred)
    
    mean_mse_te = mean_mse_te + torch.mean(vmse(pred,tsnapshot.y))

    if ttime % 4 == 0:
        H_t = None
print(mean_mse_te/(ttime+1))

tensor(0.0021, grad_fn=<DivBackward0>)


In [20]:
sum(neighbor_effects_te[0][0].values()) + model_dict['sage.lin_l.bias']

tensor([0.3499], grad_fn=<AddBackward0>)

In [21]:
target_te[0][0]

tensor([0.3499], grad_fn=<SelectBackward0>)

In [22]:
H_t = None
C_t = None
LSTM_gates_tr = []
neighbor_effects_tr = []
target_tr = []
model_explained.eval()
mean_mse_tr = 0
for ttime, tsnapshot in enumerate(train_iter):
    tsnapshot = undirect(tsnapshot)
    pred,H_t,C_t,o,f,i,na = model_explained(tsnapshot.x, tsnapshot.edge_index, H_t,C_t)

    LSTM_gates_tr.append([H_t.detach(), C_t.detach(), o.detach(), f.detach(), i.detach()])
    neighbor_effects_tr.append(na)
    target_tr.append(pred)
    
    mean_mse_tr = mean_mse_tr + torch.mean(vmse(pred,tsnapshot.y))

    if ttime % 4 == 0:
        H_t = None
print(mean_mse_tr/(ttime+1))

/Users/asadsabir/anaconda3/envs/traffic/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.0012, grad_fn=<DivBackward0>)


In [23]:
np.save('LSTM_gates_te',LSTM_gates_te)
np.save('LSTM_gates_tr',LSTM_gates_tr)

In [24]:
for i,d in enumerate(neighbor_effects_te):
    for k,v in d.items():
        for n,a in v.items():
            neighbor_effects_te[i][k][n] = float(neighbor_effects_te[i][k][n].detach().numpy())
        
for i,d in enumerate(neighbor_effects_tr):
    for k,v in d.items():
        for n,a in v.items():
            neighbor_effects_tr[i][k][n] = float(neighbor_effects_tr[i][k][n].detach().numpy())
        

/var/folders/lw/r6bk51ln09vdlk9nc13qh7280000gn/T/ipykernel_87035/2288646311.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  neighbor_effects_te[i][k][n] = float(neighbor_effects_te[i][k][n].detach().numpy())
/var/folders/lw/r6bk51ln09vdlk9nc13qh7280000gn/T/ipykernel_87035/2288646311.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  neighbor_effects_tr[i][k][n] = float(neighbor_effects_tr[i][k][n].detach().numpy())


In [ ]:
import json

with open('neighbor_effects_te.json',"w") as f:
    json.dump(neighbor_effects_te,f)

with open('neighbor_effects_tr.json',"w") as f:
    json.dump(neighbor_effects_tr,f)